In [164]:
import pandas as pd
import numpy as np
from scipy import linalg
import matplotlib.pyplot as plt
from matplotlib.dates import date2num
from pypfopt.expected_returns import mean_historical_return
from pypfopt.risk_models import CovarianceShrinkage
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import objective_functions
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices


In [165]:
all_kinds = pd.read_csv('data.csv', index_col=0)

In [166]:
all_kinds

,CTA_rate,CTA策略,S_CTA_rate,股指CTA策略,S_rate,股票策略,SF_rate,指数增强策略,MM_rate,市场中性策略
date,,,,,,,,,,
2017/12/29,-0.0013,1.0000,-0.0012,1.0000,0.0208,1.0000,0.0317,1.0000,0.0030,1.0000
2018/1/5,-0.0006,0.9987,-0.0061,0.9988,0.0130,1.0208,0.0038,1.0317,0.0006,1.0030
2018/1/12,-0.0011,0.9981,-0.0124,0.9926,-0.0086,1.0341,-0.0082,1.0357,0.0010,1.0036
2018/1/19,0.0085,0.9970,0.0024,0.9803,0.0189,1.0252,0.0222,1.0272,-0.0002,1.0046
2018/1/26,0.0126,1.0054,0.0099,0.9827,-0.0202,1.0446,-0.0405,1.0499,0.0055,1.0044
...,...,...,...,...,...,...,...,...,...,...
2021/4/23,0.0022,2.3203,-0.0041,2.0831,0.0041,1.5213,0.0034,2.2315,0.0003,1.5988
2021/4/30,0.0256,2.3255,0.0080,2.0745,-0.0196,1.5274,-0.0111,2.2391,-0.0035,1.5993
2021/5/7,0.0139,2.3849,0.0042,2.0912,0.0089,1.4975,0.0117,2.2141,0.0033,1.5936


In [167]:
cleaned_data = all_kinds.drop(['CTA策略','股指CTA策略','股票策略','指数增强策略','市场中性策略'],axis=1)
# cleaned_data = all_kinds.drop(['2021/5/21'],axis=0)

In [168]:
# cleaned_data = cleaned_data.drop(['2021/5/21'],axis = 0)
# cleaned_data = all_kinds.drop([],axis=1)
cleaned_data

,CTA_rate,S_CTA_rate,S_rate,SF_rate,MM_rate
date,,,,,
2017/12/29,-0.0013,-0.0012,0.0208,0.0317,0.0030
2018/1/5,-0.0006,-0.0061,0.0130,0.0038,0.0006
2018/1/12,-0.0011,-0.0124,-0.0086,-0.0082,0.0010
2018/1/19,0.0085,0.0024,0.0189,0.0222,-0.0002
2018/1/26,0.0126,0.0099,-0.0202,-0.0405,0.0055
...,...,...,...,...,...
2021/4/23,0.0022,-0.0041,0.0041,0.0034,0.0003
2021/4/30,0.0256,0.0080,-0.0196,-0.0111,-0.0035
2021/5/7,0.0139,0.0042,0.0089,0.0117,0.0033


In [169]:
# # # cleaned_data['date'] = date2num(cleaned_data['date'])
# # cleaned_data['CTA_rate'] = [float(x) for x in cleaned_data['CTA_rate']]
# # cleaned_data['S_CTA_rate'] = [float(x) for x in cleaned_data['S_CTA_rate']]
# # cleaned_data['S_rate'] = [float(x) for x in cleaned_data['S_rate']]
# # cleaned_data['SF_rate'] = [float(x) for x in cleaned_data['SF_rate']]
# # cleaned_data['MM_rate'] = [float(x) for x in cleaned_data['MM_rate']]
# cleaned_data_series = cleaned_data[['CTA_rate','S_CTA_rate','S_rate','SF_rate','MM_rate']].values
# # cleaned_data

In [170]:
cleaned_data.corr()

,CTA_rate,S_CTA_rate,S_rate,SF_rate,MM_rate
CTA_rate,1.000000,0.324884,-0.168580,-0.181199,-0.063154
S_CTA_rate,0.324884,1.000000,0.227584,0.247397,0.206186
S_rate,-0.168580,0.227584,1.000000,0.901369,0.457629
SF_rate,-0.181199,0.247397,0.901369,1.000000,0.538301
MM_rate,-0.063154,0.206186,0.457629,0.538301,1.000000


In [171]:
cleanrify_data = all_kinds.drop(['CTA_rate','S_CTA_rate','S_rate','SF_rate','MM_rate'],axis=1)
exp_returns = mean_historical_return(cleanrify_data)
cov_matrix = CovarianceShrinkage(cleanrify_data).ledoit_wolf()

In [172]:
exp_returns

CTA策略      2.576551
股指CTA策略    1.916739
股票策略       0.866781
指数增强策略     2.295830
市场中性策略     0.997330
dtype: float64

In [173]:
cov_matrix

,CTA策略,股指CTA策略,股票策略,指数增强策略,市场中性策略
CTA策略,0.028316,0.010502,-0.009180,-0.014126,-0.000897
股指CTA策略,0.010502,0.042595,0.015456,0.023966,0.003603
股票策略,-0.009180,0.015456,0.116652,0.147124,0.013482
指数增强策略,-0.014126,0.023966,0.147124,0.235428,0.022666
市场中性策略,-0.000897,0.003603,0.013482,0.022666,0.009971


夏普比率是投资组合的回报超过无风险利率，每单位风险（波动率）
找到最大夏普比率都投资组合
$$ SR = \frac{R_p-R_f}{\sigma} $$

In [187]:
ef = EfficientFrontier(exp_returns, cov_matrix) #有效边界
weights = ef.max_sharpe() #最大夏普比率
cleaned_weights = ef.clean_weights()
# ef.save_weights_to_file("weights.txt")  # saves to file
print(cleaned_weights)

OrderedDict([('CTA策略', 0.46198), ('股指CTA策略', 0.0571), ('股票策略', 0.0), ('指数增强策略', 0.02695), ('市场中性策略', 0.45397)])


In [188]:
ef_1 = EfficientFrontier(exp_returns, cov_matrix)
ef_1.add_objective(objective_functions.L2_reg, gamma=0.1)
w = ef_1.max_sharpe()
print(ef_1.clean_weights())
#  far fewer negligible weights than before

OrderedDict([('CTA策略', 0.44869), ('股指CTA策略', 0.23978), ('股票策略', 0.0), ('指数增强策略', 0.14413), ('市场中性策略', 0.16739)])


In [189]:
ef.portfolio_performance(verbose=True)

Expected annual return: 181.4%
Annual volatility: 9.5%
Sharpe Ratio: 18.86


(1.8143870463882001, 0.09512797553838943, 18.862874314654842)

In [190]:
ef_1.portfolio_performance(verbose=True)

Expected annual return: 211.4%
Annual volatility: 12.9%
Sharpe Ratio: 16.22


(2.113528869194155, 0.12906344401861955, 16.220928281536704)

In [191]:
latest_prices = get_latest_prices(cleanrify_data) #get latest price for each product.

In [194]:
dis_allocation = DiscreteAllocation(weights, latest_prices, total_portfolio_value=20000)
allocation, leftover = dis_allocation.lp_portfolio()
print(allocation)

NameError: Solver GLPK_MI is not installed. 